In [1]:
# Mount drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/InfoVisProject/confusionflow

/content/gdrive/MyDrive/InfoVisProject/confusionflow


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import VGG16

In [4]:
from callback import Logs

# Load Data

In [5]:
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

# Model 1

In [6]:
model_1 = Sequential(
    [
     Flatten(),
     Dense(256, activation='relu'),
     Dropout(0.2),
     Dense(128, activation='relu'),
     Dropout(0.2),
     Dense(10, activation='softmax'),
    ]
)

In [7]:
model_1_callback = Logs((x_test, y_test), loss='sparse_categorical_crossentropy')

In [8]:
model_1.compile(
    optimizer="sgd", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [9]:
model_1.fit(x_train, y_train, epochs=25, batch_size=64, callbacks=[model_1_callback])

Epoch 1/25
782/782 [==============================] - 3s 3ms/step - loss: 2.0158 - accuracy: 0.2699
Epoch 2/25
782/782 [==============================] - 2s 3ms/step - loss: 1.8412 - accuracy: 0.3435
Epoch 3/25
782/782 [==============================] - 2s 3ms/step - loss: 1.7720 - accuracy: 0.3680
Epoch 4/25
782/782 [==============================] - 2s 3ms/step - loss: 1.7222 - accuracy: 0.3877
Epoch 5/25
782/782 [==============================] - 2s 3ms/step - loss: 1.6879 - accuracy: 0.3980
Epoch 6/25
782/782 [==============================] - 2s 3ms/step - loss: 1.6596 - accuracy: 0.4143
Epoch 7/25
782/782 [==============================] - 2s 3ms/step - loss: 1.6369 - accuracy: 0.4198
Epoch 8/25
782/782 [==============================] - 2s 3ms/step - loss: 1.6114 - accuracy: 0.4317
Epoch 9/25
782/782 [==============================] - 2s 3ms/step - loss: 1.5911 - accuracy: 0.4362
Epoch 10/25
782/782 [==============================] - 2s 3ms/step - loss: 1.5699 - accuracy: 0.4441

In [10]:
model_1_callback.export('../logs', 'model_1')

# Model 2

In [11]:
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
vgg.trainable = False

In [12]:
model_2 = Sequential()
model_2.add(vgg)
model_2.add(Flatten())
model_2.add(Dense(512, activation='relu'))
model_2.add(Dense(10, activation='softmax'))

model_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 512)               262656    
                                                                 
 dense_4 (Dense)             (None, 10)                5130      
                                                                 
Total params: 14,982,474
Trainable params: 267,786
Non-trainable params: 14,714,688
_________________________________________________________________


In [13]:
model_2.compile(
    optimizer='RMSprop', loss='sparse_categorical_crossentropy', metrics=['accuracy']
)

In [14]:
model_2_callback = Logs((x_test, y_test), loss='sparse_categorical_crossentropy')

In [15]:
model_2.fit(x_train, y_train, epochs=25, batch_size=64, callbacks=[model_2_callback])

Epoch 1/25
782/782 [==============================] - 11s 13ms/step - loss: 1.3574 - accuracy: 0.5237
Epoch 2/25
782/782 [==============================] - 10s 13ms/step - loss: 1.1712 - accuracy: 0.5896
Epoch 3/25
782/782 [==============================] - 10s 13ms/step - loss: 1.0974 - accuracy: 0.6151
Epoch 4/25
782/782 [==============================] - 10s 13ms/step - loss: 1.0446 - accuracy: 0.6353
Epoch 5/25
782/782 [==============================] - 10s 13ms/step - loss: 1.0000 - accuracy: 0.6512
Epoch 6/25
782/782 [==============================] - 10s 13ms/step - loss: 0.9592 - accuracy: 0.6653
Epoch 7/25
782/782 [==============================] - 10s 13ms/step - loss: 0.9237 - accuracy: 0.6787
Epoch 8/25
782/782 [==============================] - 10s 13ms/step - loss: 0.8889 - accuracy: 0.6896
Epoch 9/25
782/782 [==============================] - 10s 13ms/step - loss: 0.8570 - accuracy: 0.6994
Epoch 10/25
782/782 [==============================] - 10s 13ms/step - loss: 0.824

In [16]:
model_2_callback.export('../logs', 'model_2')